In [1]:
#Libraries
import pandas as pd
from functions import query_to_df

# Set Pandas display options to show all rows and columns
# pd.set_option("display.max_rows", None)  # None means no limit
# pd.set_option("display.max_columns", None)  # None means no limit

In [2]:
sql = """
SELECT 
    e.*,
    v.VAX_LOT, v.VAX_MANU, v.VAX_DOSE_SERIES,
    s.SYMPTOM1, s.SYMPTOM2, s.SYMPTOM3, s.SYMPTOM4, s.SYMPTOM5
FROM event_data AS e
LEFT JOIN sample_vax_data AS v
ON e.VAERS_ID = v.VAERS_ID
LEFT JOIN sample_sympt_data AS s
ON e.VAERS_ID = s.VAERS_ID
"""
df = query_to_df(sql)
df.head()

,VAERS_ID,STATE,AGE_YRS,SEX,DIED,L_THREAT,HOSPITAL,DISABLE,RECOVD,VAX_DATE,BIRTH_DEFECT,VAX_LOT,VAX_MANU,VAX_DOSE_SERIES,SYMPTOM1,SYMPTOM2,SYMPTOM3,SYMPTOM4,SYMPTOM5
0,902418,NJ,56.0,0,0,0,0,0,1,12/15/2020,0,EH9899,PFIZER\BIONTECH,1,hypoaesthesia,injection site hypoaesthesia,,,
1,902440,AZ,35.0,0,0,0,0,0,1,12/15/2020,0,EH9899,PFIZER\BIONTECH,1,headache,,,,
2,902446,WV,55.0,0,0,0,0,0,1,12/15/2020,0,EH9899,PFIZER\BIONTECH,1,erythema,feeling hot,flushing,,
3,902464,LA,42.0,1,0,0,0,0,1,12/15/2020,0,,,,dizziness,electrocardiogram normal,hyperhidrosis,laboratory test normal,presyncope
4,902465,AR,60.0,0,0,0,0,0,0,12/15/2020,0,EH9899,PFIZER\BIONTECH,1,sensory disturbance,tremor,,,


In [3]:
df.to_csv("raw_sql_data.csv")

In [4]:
# Drop rows where 'VAX_LOT' or 'VAX_MANU' have null values
df = df.dropna(subset=['VAX_LOT', 'VAX_MANU'])

In [5]:
# Replace empty symptom fields with "None"
symptom_columns = ['SYMPTOM1', 'SYMPTOM2', 'SYMPTOM3', 'SYMPTOM4', 'SYMPTOM5']
df[symptom_columns] = df[symptom_columns].replace('', 'None')

In [6]:
# Combine SYMPTOM1 to SYMPTOM5 into a single column
df['ALL_SYMPTOMS'] = df[symptom_columns].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)


In [7]:
# Apply One-Hot Encoding on the combined symptoms
symptom_dummies = df['ALL_SYMPTOMS'].str.get_dummies(sep=',')

# Merge the one-hot encoded symptoms back into the original dataframe
df = pd.concat([df, symptom_dummies], axis=1)

In [8]:
# Display all column names
list(df.columns)

['VAERS_ID',
 'STATE',
 'AGE_YRS',
 'SEX',
 'DIED',
 'L_THREAT',
 'HOSPITAL',
 'DISABLE',
 'RECOVD',
 'VAX_DATE',
 'BIRTH_DEFECT',
 'VAX_LOT',
 'VAX_MANU',
 'VAX_DOSE_SERIES',
 'SYMPTOM1',
 'SYMPTOM2',
 'SYMPTOM3',
 'SYMPTOM4',
 'SYMPTOM5',
 'ALL_SYMPTOMS',
 ' insomnia type',
 ' visual',
 'None',
 'abdominal adhesions',
 'abdominal discomfort',
 'abdominal distension',
 'abdominal pain',
 'abdominal pain lower',
 'abdominal pain upper',
 'abdominal rigidity',
 'abnormal behaviour',
 'abnormal dreams',
 'abnormal sensation in eye',
 'abortion spontaneous',
 'abscess',
 'acne',
 'acoustic stimulation tests',
 'acoustic stimulation tests abnormal',
 'activated partial thromboplastin time',
 'activated partial thromboplastin time normal',
 'activated partial thromboplastin time shortened',
 'acute coronary syndrome',
 'acute disseminated encephalomyelitis',
 'acute generalised exanthematous pustulosis',
 'acute kidney injury',
 'acute myocardial infarction',
 'acute respiratory failure',
 

In [9]:
# Select the symptom columns
sympt_cols_df = df.iloc[:, -2036:]
# Calculate the occurrences of each symptom
symptom_counts = sympt_cols_df.sum(axis=0)

In [10]:
# Create a DataFrame to display the column names and their counts
symptom_table = pd.DataFrame({
    "Symptom": symptom_counts.index,
    "Occurrences": symptom_counts.values
})
# Sort the table by occurrences in descending order
symptom_table_sorted = symptom_table.sort_values(by="Occurrences", ascending=False)
# Display the sorted table
display(symptom_table_sorted)

,Symptom,Occurrences
2,None,16730
824,headache,4863
390,chills,3549
691,fatigue,3423
1546,pyrexia,3390
1377,pain,3250
1276,nausea,2947
546,dizziness,2753
1001,injection site pain,2363
1379,pain in extremity,2040


In [11]:
#Filter symptoms with more than 40 occurrences
symptoms_to_keep = symptom_table_sorted[symptom_table_sorted['Occurrences'] > 40]['Symptom'].tolist()
len(symptoms_to_keep)

204

In [12]:
# Get the original columns
original_cols = df.columns[:14].tolist()
original_cols

['VAERS_ID',
 'STATE',
 'AGE_YRS',
 'SEX',
 'DIED',
 'L_THREAT',
 'HOSPITAL',
 'DISABLE',
 'RECOVD',
 'VAX_DATE',
 'BIRTH_DEFECT',
 'VAX_LOT',
 'VAX_MANU',
 'VAX_DOSE_SERIES']

In [13]:
columns_to_keep = original_cols + symptoms_to_keep

In [15]:
# Define the aggregation dictionary
agg_dict = {col: "max" for col in symptoms_to_keep}  # Max for all symptom columns
agg_dict.update({col: "first" for col in df.columns if col not in symptoms_to_keep})  # Keep first for the rest
# Group by ‘vars ID’ and apply the aggregation
df_merged = df.groupby("VAERS_ID").agg(agg_dict).reset_index()
# Display the result
print(df_merged.head())

In [16]:
cleaned_sympt_df

KeyboardInterrupt: 